In [ ]:
from orphics import sehgal, maps
import healpy as hp
from pixell import utils, enmap, curvedsky, enplot, wcsutils
import os
import numpy as np

import matplotlib.pyplot as plt
import lmdb
from cosmikyu import datasets, transforms, config
from pitas import modecoupling
import random

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
data_dir = config.default_data_dir
sehgal_dir = os.path.join(data_dir, 'sehgal')
stat_dir = os.path.join(sehgal_dir, "stats")
#norm_info_file = "/home/dwhan89/workspace/cosmikyu/data/sehgal/201020_logzshrink_normalization_info_validation.npz"
#SDN = transforms.SehgalDataNormalizerScaledLogZShrink(norm_info_file)
#SDS_test = datasets.SehgalDataSet(sehgal_dir, "train141020", transforms=[SDN], dummy_label=False)
#SDS_input = datasets.SehgalDataSet(sehgal_dir, "train_tertiary191120", 
#                                   transforms=[SDN], dummy_label=False, dtype=np.float32)

norm_info_file = "/home/dwhan89/workspace/cosmikyu/data/sehgal/281220_logz_normalization_info_validation.npz"
SDN = transforms.SehgalDataNormalizerScaledLogZShrink(norm_info_file)
SDS_test = datasets.SehgalDataSet(sehgal_dir, "train281220_fromcat", transforms=[SDN], dummy_label=False,
                                    dtype=np.float64)
SDS_input = datasets.SehgalDataSet(sehgal_dir, "train_tertiary281220_fromcat", transforms=[SDN],
                                   dummy_label=False, dtype=np.float32)

plot_dir = "/home/dwhan89/scratch/outbox/cosmikyu"
def plot_path(x):
    return os.path.join(plot_dir, x)

In [ ]:
ps_file = "/home/dwhan89/workspace/cosmikyu/data/sehgal/ps_dist_train010121.npy"
nsample = len(SDS_test)
if not os.path.exists(ps_file):
    storage = np.zeros((nsample, 5))
    for i in range(len(SDS_test)):
        if i % 5000 == 0: print(i)
        storage[i,:] = np.var(SDS_test[i], axis=(-1,-2))
    np.save(ps_file, storage)
else:
    storage = np.load(ps_file)

In [ ]:
bin_edges = np.linspace(0,storage[:,2].max(),200)
bin_center = (bin_edges[:-1]+ bin_edges[1:])/2
digitized = np.digitize(storage[:,2], bin_edges)
nbin = len(bin_center)
binned = np.zeros(nbin)
for i in range(nbin):
    binned[i] = np.sum(digitized == i+1)
binned /= nsample
del digitized
#hist, binedges = np.histogram(storage[:,2], bins=100, density=False)

plt.plot(bin_center, binned)
plt.xlim(0,5)
plt.grid()

frac = 0.20
npart = int(nsample*frac)
ind = np.argpartition(storage[:,2], -npart)[-npart:]
print(np.min(storage[ind,2]))
print(npart/nsample*100)
for i in range(5):
    cidx = random.choice(ind)
    enplot.pshow(SDS_test[cidx][2])

highflux_idx = [i for i in range(nsample) if i in ind]
lowflux_idx = [i for i in range(nsample) if i not in ind]

np.save("/home/dwhan89/workspace/cosmikyu/data/sehgal/ps_dist_train010121_tszflux_sortedidx.npy", np.argsort(storage[:,2]))


In [ ]:
ps_file = "/home/dwhan89/workspace/cosmikyu/data/sehgal/ps_dist_tertiary_train010121.npy"
nsample = len(SDS_input)
if not os.path.exists(ps_file):
    storage = np.zeros((nsample, 5))
    for i in range(len(SDS_input)):
        if i % 5000 == 0: print(i)
        storage[i,:] = np.var(SDS_input[i], axis=(-1,-2))
    np.save(ps_file, storage)
else:
    storage = np.load(ps_file)

In [ ]:
bin_edges = np.linspace(0,storage[:,2].max(),200)
bin_center = (bin_edges[:-1]+ bin_edges[1:])/2
digitized = np.digitize(storage[:,2], bin_edges)
nbin = len(bin_center)
binned = np.zeros(nbin)
for i in range(nbin):
    binned[i] = np.sum(digitized == i+1)
binned /= nsample
del digitized
#hist, binedges = np.histogram(storage[:,2], bins=100, density=False)

plt.plot(bin_center, binned)
plt.xlim(0,2)
plt.grid()

frac = 0.2
npart = int(nsample*frac)
ind = np.argpartition(storage[:,2], -npart)[-npart:]
print(np.min(storage[ind,2]))
print(npart/nsample*100)
for i in range(5):
    cidx = random.choice(ind)
    enplot.pshow(SDS_test[cidx][2])
    
np.save("/home/dwhan89/workspace/cosmikyu/data/sehgal/ps_dist_tertiary_train010121_tszflux_sortedidx.npy", np.argsort(storage[:,2]))
